In [ ]:
import snowflake.connector
import dotenv
import os
import time

# Load environment variables
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")

# Snowflake connection configuration
config = {
    "account": "SFEDU02-KFB85562",
    "user": "BISON",
    "password": SNOWFLAKE_PASSWORD,
    "role": "TRAINING_ROLE",
    "warehouse": "ANIMAL_TASK_WH",
    "database": "CATCH_ME_IF_YOU_CAN",
    "schema": "PUBLIC"
}
old_count = 0
while True:
    try:
        # Open a read-only connection (retry on failure)
        conn = snowflake.connector.connect(**config)
        cur = conn.cursor()
        result = cur.execute("SELECT COUNT(*) FROM live_queries;").fetchone()
        count = result[0]
        print(f"Number of entries: {count}, new entries: {count - old_count}")
        old_count = count

        # Close the cursor and connection
        cur.close()
        conn.close()      
        
    except Exception as e:
        print(f"Reader Error: {e}")
        
    finally:
        if 'conn' in locals():
            conn.close()  # Ensure connection is closed
    
    time.sleep(1)